In [1]:
from numpy import *
class My_AdaBoostClassifier:
    def __init__(self,numIt=5000):
        self.numIt=numIt
        self.classifers=None
              
    def fit(self,X,y):
        y=2*y-1
        self.classifers,_=My_AdaBoostClassifier.adaBoostTrainDS(X,y,self.numIt)
        return self
        
    def predict(self,X):
        if self.classifers is None:
            raise Exception('The model has not yet been trained.')
        return (My_AdaBoostClassifier.adaClassify(X,self.classifers)+1)/2
        
    def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):
        #just classify the data
        retArray = ones((shape(dataMatrix)[0],1))
        if threshIneq == 'lt':
            retArray[dataMatrix[:,dimen] <= threshVal] = -1.0
        else:
            retArray[dataMatrix[:,dimen] > threshVal] = -1.0
        return retArray
    
    def buildStump(dataArr,classLabels,D):
        '''获取dataArr最佳切分列和规则，最小误差，以及切分结果
        '''
        dataMatrix = mat(dataArr)       #训练数据集X
        labelMat = mat(classLabels).T  #训练数据集y
        m,n = shape(dataMatrix)   #训练数据集的shape
        numSteps = 10.0;      #变量numSteps用于在特征的所有可能值上进行遍历
        bestStump = {};     #用于存储给定权重向量D时所得到的最佳单层决策树 的相关信息
        bestClasEst = mat(zeros((m,1)))  #初始化最优切分结果
        minError = inf #init error sum, to +infinity

        for i in range(n):#loop over all dimensions
            rangeMin = dataMatrix[:,i].min()
            rangeMax = dataMatrix[:,i].max();
            stepSize = (rangeMax-rangeMin)/numSteps

            for j in range(-1,int(numSteps)+1):#loop over all range in current dimension
                for inequal in ['lt', 'gt']: #go over less than and greater than
                    threshVal = (rangeMin + float(j) * stepSize)
                    predictedVals =My_AdaBoostClassifier.stumpClassify(dataMatrix,i,threshVal,inequal)
                    #call stump classify with i, j, lessThan
                    errArr = mat(ones((m,1)))
                    errArr[predictedVals == labelMat] = 0
                    weightedError = D.T*errArr  #calc total error multiplied by D
                    #print "split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError)
                    if weightedError < minError:
                        minError = weightedError
                        bestClasEst = predictedVals.copy()
                        bestStump['dim'] = i
                        bestStump['thresh'] = threshVal
                        bestStump['ineq'] = inequal
        return bestStump,minError,bestClasEst
        
    def adaBoostTrainDS(dataArr,classLabels,numIt=500):
        weakClassArr = []
        m = shape(dataArr)[0]
        D = mat(ones((m,1))/m)   #init D to all equal
        aggClassEst = mat(zeros((m,1)))
        for i in range(numIt):
            bestStump,error,classEst = My_AdaBoostClassifier.\
                                        buildStump(dataArr,classLabels,D)#build Stump
            alpha = float(0.5*log((1.0-error)/max(error,1e-16)))#calc alpha, throw in max(error,eps) to account for error=0
            bestStump['alpha'] = alpha  
            weakClassArr.append(bestStump)                  #store Stump Params in Array
            expon = multiply(-1*alpha*mat(classLabels).T,classEst) #exponent for D calc, getting messy
            D = multiply(D,exp(expon))                              #Calc New D for next iteration
            D = D/D.sum()
            #calc training error of all classifiers, if this is 0 quit for loop early (use break)
            aggClassEst += alpha*classEst
            #print "aggClassEst: ",aggClassEst.T
            aggErrors = multiply(sign(aggClassEst) != mat(classLabels).T,ones((m,1)))
            errorRate = aggErrors.sum()/m
            print ("total error: ",errorRate)
            if errorRate == 0.0: 
                break
        return weakClassArr,aggClassEst
    
    def adaClassify(datToClass,classifierArr):
        dataMatrix = mat(datToClass)#do stuff similar to last aggClassEst in adaBoostTrainDS
        m = shape(dataMatrix)[0]
        aggClassEst = mat(zeros((m,1)))
        for i in range(len(classifierArr)):
            classEst = My_AdaBoostClassifier.stumpClassify(dataMatrix,classifierArr[i]['dim'],\
                                     classifierArr[i]['thresh'],\
                                     classifierArr[i]['ineq'])#call stump classify
            aggClassEst += classifierArr[i]['alpha']*classEst
            #print(aggClassEst)
        return sign(aggClassEst).flatten().A[0]   

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X,y=make_classification()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
model=My_AdaBoostClassifier(numIt=15000).fit(X_train,y_train)
model.predict(X_test)

total error:  0.08571428571428572
total error:  0.08571428571428572
total error:  0.02857142857142857
total error:  0.07142857142857142
total error:  0.0


array([0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0.])

In [3]:
accuracy_score(model.predict(X_test),y_test)

0.9